# 먹을텐데 식당 추천 서비스

## 플레이리스트 수집

In [2]:
from pytubefix import Playlist

# 특정 재생목록 URL
playlist_url = 'https://www.youtube.com/playlist?list=PLuMuHAJh9g_Py_PSm8gmHdlcil6CQ9QCM'

playlist = Playlist(playlist_url)

# 재생목록의 모든 영상 출력
print(f'재생목록 제목: {playlist.title}')
for video in playlist.videos:
    print(f'{video.title}: {video.watch_url}')

재생목록 제목: 성시경의 먹을텐데
성시경의 먹을텐데 l 삼성역 해봉정육 본점 (with John K): https://youtube.com/watch?v=llHQEbY28a4
[sub] 성시경의 먹을텐데 l 중랑역 한국횟집 (with.임지연): https://youtube.com/watch?v=z9j5-nG8VoU
[sub] 성시경의 먹을텐데 l 장안동 남도보쌈파전: https://youtube.com/watch?v=GmtBOb8RJ2g
성시경의 먹을텐데 l 울산 신천지대반점: https://youtube.com/watch?v=EJ24_XjKbxY
[sub] 성시경의 먹을텐데 l 녹사평역 크리스피포크타운: https://youtube.com/watch?v=BI7EPHvf0dY
[sub] 성시경의 먹을텐데 l 평창동 이식당 (with.정우): https://youtube.com/watch?v=5ZUmZiQYZLo
[sub] 성시경의 먹을텐데 l 신림동 전라도시네마: https://youtube.com/watch?v=qbqquv_8wM0
[sub] 성시경의 먹을텐데 l 인현시장 실비식당 (with.노래 콜라보 이벤트 당첨자): https://youtube.com/watch?v=rlNH0GUBkbM
[sub] 성시경의 먹을텐데 l 부산 만우장: https://youtube.com/watch?v=zA-F9EtsRJc
[sub] 성시경의 먹을텐데 l 부산 해진아나고: https://youtube.com/watch?v=VkW3eo6C0fA
[sub] 성시경의 먹을텐데 l 속초 화진호 이선장네: https://youtube.com/watch?v=Khjn-JspNkQ
[sub] 성시경의 먹을텐데 l 남영동 옛날감자전: https://youtube.com/watch?v=1TmG1Wwb7_U
[sub] 성시경의 먹을텐데 l 반포동 삼계탕마을: https://youtube.com/watch?v=AcShBsKZW8w
[sub] 성시경의 먹을텐데 l 한남동 고다이: https://y

## 수집된 영상의 자막 스크립트 수집

- 성시경의 먹을텐데 영상은 자체 자막을 반드시 지원하고? 있는것으로 확인 됨
- 따라서 STT를 하지 않고 자막을 수집하여 메뉴 추출

In [4]:
from pytubefix import YouTube
from youtube_transcript_api import YouTubeTranscriptApi

# YouTube 객체 생성
yt = YouTube("https://youtube.com/watch?v=llHQEbY28a4")
video_id = yt.video_id  # 동영상 ID 추출

# 자막 가져오기
try:
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
    print(f"영상 제목: {yt.title}")
    for entry in transcript:
        print(f"{entry['start']}s - {entry['text']}")
except Exception as e:
    print(f"자막을 가져오는 중 오류 발생: {e}")

영상 제목: 성시경의 먹을텐데 l 삼성역 해봉정육 본점 (with John K)
3.33s - [Music]
28.96s - for for
36.96s - [Music]
68.32s - hey how are you you're wearing that
70.68s - Jackie yeah I had to you need that right
73.52s - I do need that we took a couple long
75.759s - walks just to go explore and 5 minutes
78.04s - on the road I was like glad I had it so
80.68s - thank you for this no problem okay let's
83.28s - order first I'll follow you I'm I'm
86.079s - excited see you said pork beef it's beef
89.68s - great right what is this uh it's a tea
93.52s - tea yeah
95.52s - non-alcohol so nobody knows but I well
98.159s - he told me that he quit drinking so I'll
101.079s - be the one drinking today alone thank
103.159s - you drink for me okay how this might
107.399s - work yeah oh same color it's perfect
111.119s - nobody will even know everybody knows
113.88s - Cheers Cheers welcome to
117.92s - Korea thank you and again thank you for
121.159s - the jacket okay so this is what happened
124.56s - he came Monday

## 수집된 영상의 정보 수집

In [13]:
from pytubefix import YouTube
yt = YouTube("https://youtube.com/watch?v=llHQEbY28a4")


description = yt.description.split("\n")
name = description[0]
address = description[1]

In [14]:
# 수집된 주소를 기반으로 좌표 확인 필요
address

'서울 강남구 테헤란로108길 25 대동빌딩 1층 (대치동 999-6)'

## 주소 기반으로 좌표 수집

In [15]:
import requests

# Kakao API Key
KAKAO_API_KEY = "523560117668b160f3f5dfc21200bcbc"

# Kakao Geocoding API 요청
url = "https://dapi.kakao.com/v2/local/search/address.json"
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
params = {"query": address}

response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    data = response.json()
    if data['documents']:
        location = data['documents'][0]
        latitude = location['y']
        longitude = location['x']
        print(f"위도: {latitude}, 경도: {longitude}")
    else:
        print("주소로부터 좌표를 찾을 수 없습니다.")
else:
    print("API 요청 실패:", response.status_code)


위도: 37.5077460789185, 경도: 127.066565204126


## 수집된 좌표를 지도에 표시

- streamlit에 표시해야 하므로 아래 방법은 바뀔 수 있음

In [17]:
import folium

# 위도와 경도
latitude = 37.498095  # 예시 좌표
longitude = 127.027610  # 예시 좌표

# 지도 생성
m = folium.Map(location=[latitude, longitude], zoom_start=16)
folium.Marker([latitude, longitude], popup="대동빌딩").add_to(m)

# HTML로 저장
m.save("map.html")
print("지도 파일이 'map.html'로 저장되었습니다.")

지도 파일이 'map.html'로 저장되었습니다.


## 수집된 데이터들을 db에 저장

- db는 sqlite?
- 가게이름, 주소, 좌표, 메뉴들, 후기들
    - 메뉴들, 후기들을 어떻게 묶어서 정리할 지 정해야 함.